<a href="https://colab.research.google.com/github/cedro3/ProgressiveGAN/blob/master/Search_for_Yui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Search_for_Yui


# Set up

ライブラリーインストール

In [ ]:
!pip -q install imageio
!pip -q install scikit-image
!pip install git+https://github.com/tensorflow/docs

Sampleデータのコピー

In [ ]:
!git clone https://github.com/cedro3/Sample.git

ライブラリーインポートと関数定義

In [ ]:
from absl import logging
import imageio
import PIL.Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import time
from IPython import display
from skimage import transform

try:
  from google.colab import files
except ImportError:
  pass

latent_dim = 512  # 潜在空間の次元数

# 2点間のベクトルを補完
def interpolate_hypersphere(v1, v2, num_steps):
  v1_norm = tf.norm(v1)
  v2_norm = tf.norm(v2)
  v2_normalized = v2 * (v1_norm / v2_norm)

  vectors = []
  for step in range(num_steps):
    interpolated = v1 + (v2_normalized - v1) * step / (num_steps - 1)
    interpolated_norm = tf.norm(interpolated)
    interpolated_normalized = interpolated * (v1_norm / interpolated_norm)
    vectors.append(interpolated_normalized)
  return tf.stack(vectors)

# 画像表示
def display_image(image):
  image = tf.constant(image)
  image = tf.image.convert_image_dtype(image, tf.uint8)
  return PIL.Image.fromarray(image.numpy())

# 複数の画像からアニメーション
def animate(images):
  images = np.array(images)
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images)
  return embed.embed_file('./animation.gif')

# 多点ベクトル間を補完し、画像を生成
def anime(vectors_list):
  for i in range(len(vectors_list)-1):
      vectors = interpolate_hypersphere(vectors_list[i], vectors_list[i+1], 35)  ### 50 >25
      images = progan(vectors)['default']
      if i == 0:
         anime_images = images
      else:
         anime_images = tf.concat([anime_images, images], axis=0)
  return anime_images

# ターゲット画像を生成するベクトルの探索
def find_closest_latent_vector(initial_vector, target_image):
  num_optimization_steps = 1000
  steps_per_image = 25
  images = []
  losses = []

  vector = tf.Variable(initial_vector) 
  target_image = tf.convert_to_tensor(target_image, np.float32) 
  optimizer = tf.optimizers.Adam(learning_rate=0.01)
  loss_fn = tf.losses.MeanAbsoluteError(reduction="sum")

  for step in range(num_optimization_steps):
    if (step % 20)==0:  
      print('-',end='')
    with tf.GradientTape() as tape:
      image = progan(vector.read_value())['default'][0]
      if (step % steps_per_image) == 0:
        images.append(image.numpy())
      target_image_difference = loss_fn(image, target_image[:,:,:3])
      regularizer = tf.abs(tf.norm(vector) - np.sqrt(latent_dim))
      
      loss = target_image_difference + regularizer
      losses.append(loss.numpy())
    grads = tape.gradient(loss, [vector])
    optimizer.apply_gradients(zip(grads, [vector]))
    
  return images, losses, vector.read_value()

progan 学習済みモデルのダウンロード

In [ ]:
progan = hub.load("https://tfhub.dev/google/progan-128/1").signatures['default']

# モデルを使ってみる

100個のランダムベクトルを元に、学習済みモデルで顔画像を生成

In [ ]:
tf.random.set_seed(80)  # 乱数をシード80で初期化
vector = tf.random.normal([100, latent_dim])  # ランダムベクトルを100個生成
images = progan(vector)['default']  # 学習済みモデルにランダムベクトルを入力し100個画像を生成

# 100個の画像を10×10で表示
r, c = 10, 10
fig, axs = plt.subplots(r, c, figsize=(14,14)) 
cnt = 0       
for i in range(r):
    for j in range(c):
        axs[i,j].imshow(images[cnt])
        axs[i,j].axis('off') 
        cnt += 1                
plt.show()
plt.close()

スマイル成分

In [ ]:
smile = (vector[29] + vector[53] + vector[63] + vector[99])/4  
non_smile = (vector[8] + vector[17] + vector[20] + vector[79])/4  
smile_vector = smile - non_smile  # スマイル成分の抽出
calc_vector = vector[34] + smile_vector  # vector[34]にスマイル成分を足す
image_before = display_image(progan(vector[34])['default'][0])  
image_after = display_image(progan(calc_vector)['default'][0])  
display_image(np.concatenate([image_before, image_after], axis=1))


サングラス成分




In [ ]:
glass_vector = vector[38] - vector[86]   # サングラス成分
calc_vector = vector[24] + glass_vector  # vector[24]にサングラス成分を足す 
image_before = display_image(progan(vector[24])['default'][0])  
image_after = display_image(progan(calc_vector)['default'][0])  
display_image(np.concatenate([image_before, image_after], axis=1))

ベクトル間補完のアニメーション

In [ ]:
vectors_list = [vector[29], vector[71], vector[96], vector[97], vector[53], vector[3], vector[29]]  # ベクトルの指定 
anime_images = anime(vectors_list)  # 指定されたベクトル間を補完し、画像を生成
animate(anime_images)  

# 潜在空間の画像ベクトル探索

初期ベクトル(initial_vector)の設定

In [ ]:
initial_vector = vector[53]
display_image(progan(initial_vector)['default'][0])

ターゲット画像(target_image)のアップロード\
コードを実行したら、**「ファイル選択」**ボタンを押して、Sample/target_pic/0026.jpgを選択して下さい。\
＊PCにオリジナル画像がある場合は、その画像を選択して下さい。



In [ ]:
try:
  from google.colab import files
except ImportError:
  pass

uploaded = files.upload()
image = imageio.imread(uploaded[list(uploaded.keys())[0]])
target_image = transform.resize(image, [128, 128])
display_image(target_image)

ターゲット画像に最も近い画像を生成するベクトルの探索

In [ ]:
# ターゲット画像を生成するベクトルの探索
images, loss, target_vector = find_closest_latent_vector(initial_vector, target_image)

# ロス推移表示
plt.plot(loss)
plt.ylim([0,max(plt.ylim())])
plt.show()
print()
print('loss = ', loss[-1])

# ターゲット画像とターゲットベクトルから生成した画像の表示
display_image(np.concatenate([target_image, images[-1]],  axis=1))

target_vectorの周辺の探索

In [ ]:
target = target_vector + vector/10
images = progan(target)['default']

r, c = 8, 8
fig, axs = plt.subplots(r, c, figsize=(15,15)) 
cnt = 0       
for i in range(r):
    for j in range(c):
        axs[i,j].imshow(images[cnt])
        axs[i,j].axis('off') 
        cnt += 1                
plt.show()
plt.close()